In [ ]:
addprocs(2) # vary number of concurrent processes here
@everywhere include("../src/TemporalInstanton.jl")
@everywhere include("../src/mat2tmpinst.jl")
@everywhere using TemporalInstanton

In [ ]:
cases = [
    "case9",
    "case14",
    "case24_ieee_rts",
    "case30",
    "case39",
    "case57",
    "case79",
    "case118",
    "case300"
#     "case1354pegase"
]
# wind_penetration = fill("",length(cases));

score_output = Array(Vector{Float64},0)
sec_elapsed = []
bytes_alloc = []
sec_in_gc = []

for i in 1:length(cases)
    case = cases[i]

    inputData = mat2tmpinst(case; fill_default=true);

    #wind_penetration[i] = "$(round(sum(Rp)/(sum(Gp)+sum(Rp))*100,1))%"

    # Thermal model parameters:
    inputData.Tamb = 35. # C
    inputData.T0 = 60. #46. # initial line steady-state temp

    inputData.time_values = 0:30:300 # five minutes in 30-sec steps
    inputData.int_length = 300. # seconds = 5 min

    Gp,Dp,Rp = (inputData.G0, inputData.D0, inputData.R0)
    inputData.G0 = [Gp;Gp;Gp;Gp;Gp;Gp]
    inputData.D0 = [Dp;Dp;Dp;Dp;Dp;Dp]
    inputData.R0 = [Rp;1.1*Rp;1.2*Rp;1.1*Rp;Rp;0.9*Rp]

    n = length(inputData.k)
    nr = length(inputData.Ridx)
    T = convert(Int64,length(inputData.G0)/n)
    
    timed_results = @timed solve_temporal_instanton(inputData);
    results = timed_results[1]
    outputData = process_instanton_results(results,n,nr,T,return_as_type=true);
    push!(score_output,outputData.score)
    push!(sec_elapsed,timed_results[2])
    push!(bytes_alloc,timed_results[3])
    push!(sec_in_gc,timed_results[4])
end

lines_processed = []
for score_vec in score_output
    push!(lines_processed, sum(score_vec.!=Inf))
end